In [1]:
!./perlin-master/target/debug/perlin #Generates perlin image!! Look at read me for specific variatin

In [2]:
!./perlin-master/target/debug/perlin -o out2
!mv out2.png ./images

In [15]:
from math import floor 

In [17]:
#Stroke combination 
def image_combination(image1,image2):
    background = Image.open(image1)
    overlay = Image.open(image2)

    background = background.convert("RGBA")
    overlay = overlay.convert("RGBA")

    new_img = Image.blend(background, overlay, 0.5)
    new_img.save("./images/painting.png","PNG")

In [78]:
class MoodImage:
    def __init__(self,state_num,perlinOrManual):
        self.perlinOrManual = perlinOrManual
        image = self.moodProcess(state_num) #TODO 
        self.painting = image

    def getPasscode(self):
        self.passcode = input('Type in your password for sudo command')
    
    def moodProcess(self,state_num):
        self.processState(state_num)
        if self.perlinOrManual == 1:
            self.perlinSimulation("TODO")
        else:
            self.manualStrokes("TODO")

    def processState(self,state_num):
        if self.perlinOrManual == 1:
            numMoods = 3
        else:
            numMoods = 8
        self.mood_num = floor(state_num/numMoods) #Mood <- In Perlin Evolution, limited to 3 colors 
        self.stroke_num = state_num % numMoods

    
    #This is the perlin simluator <- procedurally generated perlin creator 
    def perlinSimulation(self,temp):
        print(self.mood_num)
        print(self.stroke_num)
        self.stroke_num += 1
        with open('sample.txt', 'w') as f:
            f.write(str(self.mood_num) + '\n')    
            f.write(str(self.stroke_num) + '\n')  


    #This is the manual strokes come hackathon participants
    def manualStrokes(self,temp):
        print(temp)

    def addStroke(self,state_num):
        self.processState(state_num)


In [79]:
QuantumPainter = MoodImage(1,1)

0
1
./perlin-master/target/debug/perlin: line 1: 2: command not found


In [37]:
passcode = input('Type in your password for sudo command')

In [51]:
!echo Nomadic@1991 | sudo -S ./perlin-master/target/debug/perlin -o out2

Password:^C


In [53]:
!./perlin-master/target/debug/perlin

In [62]:
!./perlin-master/target/debug/perlin -c 0 -f 1 -o testPerlin 

In [70]:
!./perlin-master/target/debug/perlin -c 1 -f 1 -o testPerlin

^C


In [81]:
d = [1,2]
with open('sample.txt', 'w') as f:
    f.write(str(d[0]) + '\n')    
    f.write(str(d[1]) + '\n')  
 